# Matrix Row and Column Operations in RustLab Math

This notebook demonstrates the new row/column extraction and vector-to-matrix conversion capabilities added to rustlab-math. These methods are essential for machine learning and linear algebra operations, especially in linear regression implementations.

## Setup and Imports

First, let's import the necessary dependencies and create some sample data.

In [2]:
:dep rustlab-math = { path = ".." }
:dep rustlab-linearalgebra = { path = "../../rustlab-linearalgebra" }

use rustlab_math::{ArrayF64, VectorF64, array64, vec64};
use rustlab_math::BasicStatistics;
use rustlab_linearalgebra::BasicLinearAlgebra;

println!("✅ Dependencies loaded successfully!");

✅ Dependencies loaded successfully!


## 1. Column Extraction - Feature Analysis

Column extraction is fundamental for analyzing features in datasets. Each column typically represents a feature in machine learning contexts.

In [3]:
{
    // Create a sample dataset: 5 samples × 3 features
    let dataset = array64![
        [1.0, 2.0, 3.0],   // Sample 1
        [4.0, 5.0, 6.0],   // Sample 2
        [7.0, 8.0, 9.0],   // Sample 3
        [10.0, 11.0, 12.0], // Sample 4
        [13.0, 14.0, 15.0]  // Sample 5
    ];
    
    println!("Dataset shape: {:?}\n", dataset.shape());
    
    // Extract each feature column
    for j in 0..dataset.ncols() {
        let feature = dataset.col(j).unwrap();
        let mean = feature.mean();
        let std = feature.std(None);
        
        println!("Feature {}: {:?}", j, feature.to_slice());
        println!("  Mean: {:.2}, Std Dev: {:.2}\n", mean, std);
    }
}

Dataset shape: (5, 3)

Feature 0: [1.0, 4.0, 7.0, 10.0, 13.0]
  Mean: 7.00, Std Dev: 4.74

Feature 1: [2.0, 5.0, 8.0, 11.0, 14.0]
  Mean: 8.00, Std Dev: 4.74

Feature 2: [3.0, 6.0, 9.0, 12.0, 15.0]
  Mean: 9.00, Std Dev: 4.74



()

## 2. Row Extraction - Sample Analysis

Row extraction allows us to analyze individual samples or observations in our dataset.

In [4]:
{
    // Create a dataset with different types of samples
    let samples = array64![
        [0.5, 1.2, 0.8, 0.3],  // Low values
        [5.5, 6.2, 5.8, 6.3],  // Medium values
        [9.5, 10.2, 9.8, 10.3] // High values
    ];
    
    println!("Analyzing individual samples:\n");
    
    for i in 0..samples.nrows() {
        let sample = samples.row(i).unwrap();
        let sum = sample.sum_elements();
        let mean = sample.mean();
        let l2_norm = (&sample ^ &sample).sqrt();  // L2 norm
        
        println!("Sample {}: {:?}", i, sample.to_slice());
        println!("  Sum: {:.2}, Mean: {:.2}, L2 Norm: {:.2}\n", sum, mean, l2_norm);
    }
}

Analyzing individual samples:

Sample 0: [0.5, 1.2, 0.8, 0.3]
  Sum: 2.80, Mean: 0.70, L2 Norm: 1.56

Sample 1: [5.5, 6.2, 5.8, 6.3]
  Sum: 23.80, Mean: 5.95, L2 Norm: 11.92

Sample 2: [9.5, 10.2, 9.8, 10.3]
  Sum: 39.80, Mean: 9.95, L2 Norm: 19.91



()

## 3. Vector to Matrix Conversion for Linear Algebra

Converting vectors to column matrices is essential for matrix operations in linear algebra, particularly when solving linear systems.

In [5]:
{
    // Create a vector
    let v = vec64![1.0, 2.0, 3.0, 4.0];
    println!("Original vector: {:?}", v.to_slice());
    println!("Vector length: {}\n", v.len());
    
    // Convert to column matrix
    let col_matrix = ArrayF64::from_vector_column(&v);
    println!("Column matrix shape: {:?}", col_matrix.shape());
    println!("Column matrix contents:");
    for i in 0..col_matrix.nrows() {
        println!("  [{:.1}]", col_matrix[(i, 0)]);
    }
    
    // Extract back to vector
    let extracted = col_matrix.to_vector_column();
    println!("\nExtracted vector: {:?}", extracted.to_slice());
    println!("Vectors are equal: {}", v.to_slice() == extracted.to_slice());
}

Original vector: [1.0, 2.0, 3.0, 4.0]
Vector length: 4

Column matrix shape: (4, 1)
Column matrix contents:
  [1.0]
  [2.0]
  [3.0]
  [4.0]

Extracted vector: [1.0, 2.0, 3.0, 4.0]
Vectors are equal: true


()

## 4. Practical Example: Solving Linear Systems

Let's demonstrate how these operations are used in solving the normal equations for linear regression: (X'X)β = X'y

In [6]:
{
    // Design matrix X (with intercept column) and target vector y
    let X = array64![
        [1.0, 1.0],   // Sample 1: intercept=1, feature=1
        [1.0, 2.0],   // Sample 2: intercept=1, feature=2
        [1.0, 3.0],   // Sample 3: intercept=1, feature=3
        [1.0, 4.0]    // Sample 4: intercept=1, feature=4
    ];
    
    // Target values: y = 2 + 3*x (true relationship)
    let y = vec64![5.0, 8.0, 11.0, 14.0];
    
    println!("Design matrix X:");
    for i in 0..X.nrows() {
        let row = X.row(i).unwrap();
        println!("  {:?}", row.to_slice());
    }
    println!("\nTarget vector y: {:?}\n", y.to_slice());
    
    // Solve normal equations: β = (X'X)^(-1)X'y
    let Xt = X.transpose();
    let XtX = &Xt ^ &X;
    
    // Convert y to column matrix for matrix multiplication
    let y_matrix = ArrayF64::from_vector_column(&y);
    let Xty_matrix = &Xt ^ &y_matrix;
    
    // Solve for coefficients
    let XtX_inv = XtX.inv().unwrap();
    let beta_matrix = &XtX_inv ^ &Xty_matrix;
    
    // Extract coefficients back to vector
    let beta = beta_matrix.to_vector_column();
    
    println!("Estimated coefficients:");
    println!("  Intercept (β₀): {:.2}", beta[0]);
    println!("  Slope (β₁): {:.2}", beta[1]);
    println!("\nTrue values were: β₀=2.0, β₁=3.0");
}

Design matrix X:
  [1.0, 1.0]
  [1.0, 2.0]
  [1.0, 3.0]
  [1.0, 4.0]

Target vector y: [5.0, 8.0, 11.0, 14.0]

Estimated coefficients:
  Intercept (β₀): 2.00
  Slope (β₁): 3.00

True values were: β₀=2.0, β₁=3.0


()

## 5. Zero-Copy Views for Performance

For large datasets, zero-copy views provide efficient access without memory allocation.

In [7]:
{
    // Create a larger dataset
    let large_data = ArrayF64::ones(1000, 100);
    println!("Large dataset shape: {:?}", large_data.shape());
    println!("Memory size: ~{:.2} MB\n", (1000 * 100 * 8) as f64 / 1_000_000.0);
    
    // Zero-copy column view - instant access
    let col_view = large_data.col_view(50).unwrap();
    println!("Column view shape: {:?}", col_view.shape());
    println!("View created with zero memory allocation!\n");
    
    // Zero-copy row view
    let row_view = large_data.row_view(500).unwrap();
    println!("Row view shape: {:?}", row_view.shape());
    
    // Convert view to owned Vector only when needed
    let owned_col = col_view.col(0).unwrap();
    println!("\nOwned column extracted, length: {}", owned_col.len());
    
    // Performance comparison message
    println!("\n📊 Performance Note:");
    println!("  - col_view(): O(1) time, no allocation");
    println!("  - col(): O(n) time, allocates memory");
    println!("  Use views for temporary access, owned for storage!");
}

Large dataset shape: (1000, 100)
Memory size: ~0.80 MB

Column view shape: (1000, 1)
View created with zero memory allocation!

Row view shape: (1, 100)

Owned column extracted, length: 1000

📊 Performance Note:
  - col_view(): O(1) time, no allocation
  - col(): O(n) time, allocates memory
  Use views for temporary access, owned for storage!


()

## 6. Feature Normalization Example

A common preprocessing step in machine learning is feature normalization (standardization).

In [8]:
{
    // Create a dataset with features at different scales
    let data = array64![
        [1.0, 100.0, 0.001],
        [2.0, 200.0, 0.002],
        [3.0, 300.0, 0.003],
        [4.0, 400.0, 0.004],
        [5.0, 500.0, 0.005]
    ];
    
    println!("Original data:");
    for i in 0..data.nrows() {
        let row = data.row(i).unwrap();
        println!("  {:?}", row.to_slice());
    }
    
    // Normalize each feature (column)
    let mut normalized = ArrayF64::zeros(data.nrows(), data.ncols());
    
    for j in 0..data.ncols() {
        let feature = data.col(j).unwrap();
        let mean = feature.mean();
        let std = feature.std(None);
        
        println!("\nFeature {}: mean={:.3}, std={:.3}", j, mean, std);
        
        // Normalize: (x - mean) / std
        for i in 0..data.nrows() {
            normalized[(i, j)] = (data[(i, j)] - mean) / std;
        }
    }
    
    println!("\nNormalized data:");
    for i in 0..normalized.nrows() {
        let row = normalized.row(i).unwrap();
        print!("  [");
        for (idx, val) in row.to_slice().iter().enumerate() {
            if idx > 0 { print!(", "); }
            print!("{:6.3}", val);
        }
        println!("]");
    }
    
    // Verify normalization
    println!("\nVerification (each feature should have mean≈0, std≈1):");
    for j in 0..normalized.ncols() {
        let feature = normalized.col(j).unwrap();
        println!("  Feature {}: mean={:.6}, std={:.6}", j, feature.mean(), feature.std(None));
    }
}

Original data:
  [1.0, 100.0, 0.001]
  [2.0, 200.0, 0.002]
  [3.0, 300.0, 0.003]
  [4.0, 400.0, 0.004]
  [5.0, 500.0, 0.005]

Feature 0: mean=3.000, std=1.581

Feature 1: mean=300.000, std=158.114

Feature 2: mean=0.003, std=0.002

Normalized data:
  [-1.265, -1.265, -1.265]
  [-0.632, -0.632, -0.632]
  [ 0.000,  0.000,  0.000]
  [ 0.632,  0.632,  0.632]
  [ 1.265,  1.265,  1.265]

Verification (each feature should have mean≈0, std≈1):
  Feature 0: mean=0.000000, std=1.000000
  Feature 1: mean=0.000000, std=1.000000
  Feature 2: mean=0.000000, std=1.000000


()

## 7. Performance Comparison: Owned vs Views

Let's demonstrate when to use owned extraction vs zero-copy views.

In [9]:
{
    use std::time::Instant;
    
    // Create a large matrix for performance testing
    let big_matrix = ArrayF64::ones(10000, 100);
    println!("Test matrix: {} × {}", big_matrix.nrows(), big_matrix.ncols());
    println!("Memory: ~{:.2} MB\n", (10000 * 100 * 8) as f64 / 1_000_000.0);
    
    // Test 1: Owned column extraction
    let start = Instant::now();
    for _ in 0..100 {
        let _col = big_matrix.col(50).unwrap();  // Allocates memory
    }
    let owned_time = start.elapsed();
    
    // Test 2: View column extraction
    let start = Instant::now();
    for _ in 0..100 {
        let _col_view = big_matrix.col_view(50).unwrap();  // Zero-copy
    }
    let view_time = start.elapsed();
    
    println!("Performance Results (100 iterations):");
    println!("  Owned col():     {:?}", owned_time);
    println!("  View col_view(): {:?}", view_time);
    println!("  Speedup:         {:.1}x faster\n", 
             owned_time.as_nanos() as f64 / view_time.as_nanos() as f64);
    
    println!("📌 Guidelines:");
    println!("  • Use col_view() for temporary access (reading)");
    println!("  • Use col() when you need to store or modify");
    println!("  • Use col_view() in loops to avoid allocations");
    println!("  • Use col() when passing data to other functions");
}

Test matrix: 10000 × 100
Memory: ~8.00 MB

Performance Results (100 iterations):
  Owned col():     1.705371ms
  View col_view(): 0ns
  Speedup:         infx faster

📌 Guidelines:
  • Use col_view() for temporary access (reading)
  • Use col() when you need to store or modify
  • Use col_view() in loops to avoid allocations
  • Use col() when passing data to other functions


()

## 8. Complete Linear Regression Example

Let's put it all together with a complete linear regression implementation using our new methods.

In [10]:
{
    // Generate synthetic dataset: y = 3 + 2*x1 - 1.5*x2 + noise
    let X_data = array64![
        [1.0, 2.0, 1.0],
        [1.0, 3.0, 2.0],
        [1.0, 4.0, 1.5],
        [1.0, 5.0, 3.0],
        [1.0, 6.0, 2.5],
        [1.0, 7.0, 4.0]
    ];
    
    let y_true = vec64![6.5, 7.0, 9.25, 8.5, 10.75, 11.0];
    
    println!("Dataset:");
    println!("  Samples: {}", X_data.nrows());
    println!("  Features: {} (including intercept)\n", X_data.ncols());
    
    // Display feature statistics
    println!("Feature Statistics:");
    for j in 0..X_data.ncols() {
        let feature = X_data.col(j).unwrap();
        let name = match j {
            0 => "Intercept",
            1 => "Feature 1",
            2 => "Feature 2",
            _ => "Feature N"
        };
        println!("  {}: mean={:.2}, std={:.2}", 
                 name, feature.mean(), feature.std(None));
    }
    
    // Fit linear regression using normal equations
    let Xt = X_data.transpose();
    let XtX = &Xt ^ &X_data;
    let y_matrix = ArrayF64::from_vector_column(&y_true);
    let Xty = &Xt ^ &y_matrix;
    
    let XtX_inv = XtX.inv().unwrap();
    let beta_matrix = &XtX_inv ^ &Xty;
    let beta = beta_matrix.to_vector_column();
    
    println!("\nEstimated Coefficients:");
    println!("  β₀ (intercept): {:.3}", beta[0]);
    println!("  β₁ (feature 1): {:.3}", beta[1]);
    println!("  β₂ (feature 2): {:.3}", beta[2]);
    
    // Make predictions
    let y_pred_matrix = &X_data ^ &beta_matrix;
    let y_pred = y_pred_matrix.to_vector_column();
    
    // Calculate R²
    let residuals = &y_true - &y_pred;
    let ss_res = &residuals ^ &residuals;
    let y_mean = y_true.mean();
    let y_centered = &y_true - y_mean;
    let ss_tot = &y_centered ^ &y_centered;
    let r_squared = 1.0 - ss_res / ss_tot;
    
    println!("\nModel Performance:");
    println!("  R² Score: {:.4}", r_squared);
    
    println!("\nPredictions vs Actual:");
    for i in 0..y_true.len() {
        println!("  Sample {}: Actual={:.2}, Predicted={:.2}, Error={:.3}",
                 i, y_true[i], y_pred[i], (y_true[i] - y_pred[i]).abs());
    }
}

Dataset:
  Samples: 6
  Features: 3 (including intercept)

Feature Statistics:
  Intercept: mean=1.00, std=0.00
  Feature 1: mean=4.50, std=1.87
  Feature 2: mean=2.33, std=1.08

Estimated Coefficients:
  β₀ (intercept): 4.613
  β₁ (feature 1): 1.542
  β₂ (feature 2): -1.164

Model Performance:
  R² Score: 0.9846

Predictions vs Actual:
  Sample 0: Actual=6.50, Predicted=6.53, Error=0.032
  Sample 1: Actual=7.00, Predicted=6.91, Error=0.091
  Sample 2: Actual=9.25, Predicted=9.03, Error=0.217
  Sample 3: Actual=8.50, Predicted=8.83, Error=0.328
  Sample 4: Actual=10.75, Predicted=10.95, Error=0.202
  Sample 5: Actual=11.00, Predicted=10.75, Error=0.253


()

## Summary

The new row/column operations in rustlab-math provide:

1. **Owned Extraction** (`col()`, `row()`): Creates new Vectors with copied data
2. **Zero-Copy Views** (`col_view()`, `row_view()`): Efficient temporary access
3. **Vector-Matrix Conversion** (`from_vector_column()`, `to_vector_column()`): Essential for linear algebra

### Key Takeaways:

- Use **views** for temporary access and iteration (no memory allocation)
- Use **owned** extraction when you need to store or modify data
- **Vector-to-matrix** conversion enables seamless linear algebra operations
- These operations are fundamental for implementing machine learning algorithms

### Performance Guidelines:

- `col_view()`/`row_view()`: O(1) time, zero allocation
- `col()`/`row()`: O(n) time, allocates memory
- Choose based on your use case: temporary vs permanent access